In [4]:
#Importa as bibliotecas

import init


from analisys.bkt_analisys import *
from analisys.data_analisys import *
from analisys.graph_analisys import *
from analisys.optmizer_analisys import *

import numpy as np
import pandas as pd
import seaborn as sb
import random
import os
import plotly.express as px
from backend.resampler import *
from backend.mt5_tools import *

from statsmodels.tsa.stattools import adfuller
from analisys.graph_analisys import moving_avg

from scipy.optimize import rosen, differential_evolution



In [5]:
start_data = '2021.04.04'
end_data = '2021.11.01'

name_x = 'PETR3'
name_y = 'PETR4'

timeframe = 5


s_spread = 0
l_spread = -1
tp_short = 0
tp_long = 0
tp_return = 1500
stop_return = -289 
stop_number = 43
period = 44
p_size = 5000

hist_x = mt5_singlehistoricaldata(name_x,timeframe,start_data, end_data)
hist_y = mt5_singlehistoricaldata(name_y,timeframe,start_data, end_data)


In [6]:
result = runbkt(hist_x, hist_y, name_x, name_y, period, s_spread, l_spread, tp_short, tp_long, stop_number, 50 ,p_size)


OverflowError: cannot convert float infinity to integer

In [ ]:
result.keys()

In [ ]:
result['positions_list'][0]['profit']

bar_profit = []

for i in result['positions_list']:
    bar_profit.append(i['profit'])

print(bar_profit)

In [ ]:
import seaborn as sns

fig, ax1 = plt.subplots()
ax1.set_ylabel('retorno (R$)')
plt.xlabel('Operações')

ax1.bar(x = range(len(result['positions_list'].invert())),height= bar_profit, alpha = 0.7)
ax2 = ax1.twinx()
ax2.plot(np.cumsum(result['returns']), 'go-', linewidth=3)
ax2.set_ylabel('lucro (R$)')
plt.show()



In [ ]:
breakpoint()

In [ ]:
from datetime import datetime
t = result['positions_list'][0]['time'][0]
new_t = datetime.strptime(t, '%Y-%m-%d %H:%M:%S') 


In [ ]:
new_t.day

In [ ]:
result.keys()

In [ ]:
opt_param = 'profit'

def simplebkt(  symbol_x,
                symbol_y,
                name_x,
                name_y,
                _period,
                s_spread,
                l_spread,
                tp_s,
                tp_l,
                tp_r,
                stop_n,
                stop_r,
                pos_size=5000):

    #fname = '{}x{}x{}{}{}.json'.format(name_x, name_y, datetime.datetime.now().hour, datetime.datetime.now().minute, datetime.datetime.now().second)

    dic = runbkt(symbol_x, symbol_y, name_x, name_y, _period,s_spread, l_spread, tp_s, tp_l, stop_n, stop_r, pos_size)
    return dic['profit']

def treino(x, name_x, name_y, df_treino):

    _p, shortspread, longspread, takeprofitshort, takeprofitlong, takeprofitreturn, stopnumber, stopreturn = x
    name_x, name_y, df_treino = arg
    r = simplebkt(df_treino[name_x], df_treino[name_y], name_x, name_y, _p, shortspread, longspread, takeprofitshort, takeprofitreturn, takeprofitlong, stopnumber, stopreturn)
    return -r

result_opttrain = forestbkt(name_x, name_y, treino, hist_test, n=40, opt_param='profit', pos_size=5000, space=space_param)
a,b,c,d,e,f,g,h = param

forest = runbkt(df_test[name_x], df_test[name_y], name_x, name_y, a,b,c,d,e,f,g,h, 5000)

print(param)

In [ ]:
#Roda otimização de parametros
bounds = space_param
arg = [name_x, name_y, df_treino]




In [ ]:
result_differential = differential_evolution(treino, bounds, args = [name_x, name_y, df_treino], updating='deferred',
                                workers=8, disp=True, maxiter = 10)




In [ ]:
a,b,c,d,e,f,g,h = result_differential['x']

res = runbkt(df_test[name_x], df_test[name_y], name_x, name_y, a, b, c, d, e, f, g, h, p_size = 300)

print(forest['returns'], res['returns'], result_differential['x'])